In [ ]:
import os

from exif import Image
import folium

from tqdm import tqdm

In [ ]:
scene = '0015'
img_dir = f'/Volumes/Extreme_SSD/MegaDepth/scenes/{scene}/images/'

img_list = os.listdir(img_dir)

exif_list = []
for img in tqdm(img_list):

    try:
        img_path = os.path.join(img_dir, img)
        with open(img_path, 'rb') as src:
            img = Image(src)

        if img.has_exif and img.gps_longitude:
            exif_list.append(img_path)
    except:
        pass

len(exif_list)

In [ ]:
def decimal_coords(coords, ref):
 decimal_degrees = coords[0] + coords[1] / 60 + coords[2] / 3600
 if ref == "S" or ref == "W":
     decimal_degrees = -decimal_degrees
 return decimal_degrees

def image_coordinates(image_path):
    with open(image_path, 'rb') as src:
        img = Image(src)
    if img.has_exif:
        try:
            img.gps_longitude
            coords = (decimal_coords(img.gps_latitude,
                      img.gps_latitude_ref),
                      decimal_coords(img.gps_longitude,
                      img.gps_longitude_ref))
        except AttributeError:
            print('No Coordinates')
            coords = None
        except NotImplementedError:
            print('No Coordinates')
            coords = None
    else:
        print('The Image has no EXIF information')
        return None

    return coords

In [ ]:
# plot location on map
def plot_map(coords: list, paths: list):
    map = folium.Map(location=coords[0], zoom_start=15)
    for coord, path in zip(coords, paths):
        folium.Marker(coord, popup=path).add_to(map)
    return map

coords = [image_coordinates(img_path) for img_path in exif_list]
coords = [coord for coord in coords if coord is not None]
exif_list = [img_path for img_path, coord in zip(exif_list, coords) if coord is not None]
plot_map(coords, exif_list)